## Capstone Project
Last week we introduced the [challenging Acute Myeloid Leukemia dataset](https://www.synapse.org/#!Synapse:syn2455683/wiki/64007) to test your ML skills. Let us take a few minutes before we begin today to touch base. To recap we indicated that you will have to request access to this data through the link provided and follow directions to the 'request access' form. At this point you should ideally have the data and have begun formatting.

Last week we outlined the analysis as follows.
+ ***Format*** the data with observations in rows and features in columns
+ ***Manually exclude data*** like book keeping variables
+ ***Normalize*** Data
+ ***Treat missing*** Data
+ Choose and employ a ***Data Partitioning*** scheme
+ Do ***Feature selection*** for Clinical covariates and ***Dimension Reduction*** for omics data.
+ ***Choose a model*** with the highest performance potential
+ ***Tune your model***
+ Caclulate ***Performance metrics***
+ Report ***Model Predicitions***
+ ***Interpret*** results


## Synapse API
The `synapseclient` package provides an interface to [Synapse](http://www.synapse.org/), a collaborative workspace for reproducible data intensive research projects, providing support for:

 + integrated presentation of data, code and text

 + fine grained access control

 + [provenance tracking](https://python-docs.synapse.org/build/html/Activity.html)

The `synapseclient` package lets you communicate with the cloud-hosted Synapse service to access data and create shared data analysis projects from within Python scripts or at the interactive Python console. Other Synapse clients exist for [R](https://r-docs.synapse.org/), [Java](https://github.com/Sage-Bionetworks/Synapse-Repository-Services/tree/develop/client/synapseJavaClient), and the [web](https://www.synapse.org/). The Python client can also be used from the [command line](https://python-docs.synapse.org/build/html/CommandLineClient.html).

If you’re just getting started with Synapse, have a look at the Getting Started guides for [Synapse](https://docs.synapse.org/articles/getting_started.html).

In [ ]:
!pip3 install --upgrade synapseclient

### Connecting to Synapse
To use Synapse, you’ll need to [register](https://www.synapse.org/register) for an account. The Synapse website can authenticate using a Google account, but you’ll need to take the extra step of creating a Synapse password to use the programmatic clients.

Once that’s done, you’ll be able to load the library, create a [Synapse](https://python-docs.synapse.org/build/html/index.html#synapseclient.Synapse) object and login:

Valid combinations of login() arguments:
+ email/username and password
+ email/username and apiKey (Base64 encoded string)
+ authToken
+ sessionToken (DEPRECATED)

If no login arguments are provided or only username is provided, login() will attempt to log in using
information from these sources (in order of preference):
1. User’s personal access token from environment the variable: SYNAPSE_AUTH_TOKEN
2. `.synapseConfig` file (in user home folder unless configured otherwise)
3. cached credentials from previous login() where rememberMe=True was passed as a parameter


### Login the safe way
The safest login method utilizes an api key stored locally in your home directory. You can get your apikey from the account settings on your Synapse dashboard. A copy of the default .synapseConfig template can also be obtained [here](https://raw.githubusercontent.com/Sage-Bionetworks/synapsePythonClient/develop/synapseclient/.synapseConfig).


### Login the easy way
The easiest way to login is to enter your email and password in plain text **yikes!!** and use the `rememberMe=True` option. This will cache your credentials on your local machine. Then, immediately remove your plain text credentials from the notebook. Next time you run the notebook it should look up your saved credentials. 

In [ ]:
import synapseclient

#create synapse object
syn = synapseclient.Synapse()

#login the easy
#syn.login(email='my_username', password='my_password', rememberMe=True)

#login the safe way
#syn.login()

In [ ]:
#Check if my credentials were saved properly
syn.logout()
syn.login()

### Data Download  & Caching
Files can be downloaded in bulk using the syncFromSynapse function found in the synapseutils helper package. This function crawls all the subfolders of the project or folder that you specify and retrieves all the files that have not been downloaded. By default, the files will be downloaded into your synapseCache, but a different download location can be specified with the path parameter. If you do download to a location out side of synapseCache, this function will also create a tab-delimited manifest of all the files along with their metadata (path, provenance, annotations, etc).

In [ ]:
import synapseutils

# download Leukemia datasets from Synapse to the data folder distributed with this notebook
files = synapseutils.syncFromSynapse(syn, entity='syn2488690', path='./data') 

You'll find 4 files downloaded to the data folder distributed with this notebook.
+ scoringData-extendedChallenge-noChemo-release.xlsx
+ trainingData-extendedChallenge-noChemo-release.xlsx
+ scoringData-release.csv
+ trainingData-release.csv

and a manifest file
+ SYNAPSE_METADATA_MANIFEST.tsv
containing the downloaded file paths and some other metadata not important for us now.

We will be working with the `trainingData-release.csv` and `scoringData-release.csv` files.

In [ ]:
#read data as pandas dataframe and display head
df = pd.read_csv('data/trainingData-release.csv')
df.head()

# Module 10 - Leukemia project  week 2

Today we will complete the kaggle Leukemia in class analysis. Continue on your own with the Synapse Dream Challenge Leukemia dataset analysis. 


We will be extending last week's notebook where we processed the gene expression data uploaded from kaggle. To quickly recap the kaggle dataset was aquired in 1999 and later [published in Science](https://doi.org/10.1126/science.286.5439.531). The paper demonstrated how new cases of cancer could be classified by gene expression monitoring (via DNA microarray) and thereby provided a general approach for identifying new cancer classes and assigning tumors to known classes. The data was used to classify patients with acute myeloid leukemia (AML) and acute lymphoblastic leukemia (ALL). Our excersice is to develop a model that discriminates between AML and ALL patients based only on the this gene expression data and compare our model with the published results of the 1999 paper.

## Setup
Let's get all the requirements sorted before we move on to the excercise.  

In [ ]:
# Requirements
!pip install --upgrade ipykernel
!pip install kaggle
!pip install pandas
!pip install tableone
!pip install numpy
!pip install matplotlib
!pip install scipy
!pip install seaborn

# Globals
seed = 1017

#imports
import kaggle
import pandas as pd
import seaborn as sns
from tableone import TableOne
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats


#magic
%matplotlib inline

## Loading the data via kaggle API
The leukemia data set was sourced from kaggle. 
To download the data directly from [kaggle](https://kaggle.com) you will need to have a kaggle account. **It's free.** Once you create your kaggle account you can generate an API token. After you log in you should see a circular account icon in the upper-right of any kaggle page. Clicking on your account icon will open a right-sidebar where you can select "Account" to edit your account. Scroll down to the API section and click on the "create new api token" button. An API token should automatically download and a prompt will also appear telling you which directory to put this token so python knows where find it. For MacOS users this location is "~/.kaggle/kaggle.json". Once you have done this modify the code below to download the dataset to the `data` folder distributed with this notebook.

In [ ]:
#log in to kaggle using your api token
kaggle.api.authenticate()

#path relative to this notebook to put the data
datadir = 'data'

#name of the dataset on kaggle
dataset = 'crawford/gene-expression'

#downlaod the data
kaggle.api.dataset_download_files(dataset, path=datadir, unzip=True)

## Loading the data
There are two datasets containing the initial (training, 38 samples) and independent (test, 34 samples) datasets used in the paper. These datasets contain measurements corresponding to ALL and AML samples from Bone Marrow and Peripheral Blood. Intensity values have been re-scaled such that overall intensities for each chip are equivalent.

In [ ]:
# download the data as a pandas dataframe
labels = pd.read_csv('data/actual.csv', index_col = 'patient')
test = pd.read_csv('data/data_set_ALL_AML_independent.csv')
train = pd.read_csv('data/data_set_ALL_AML_train.csv')

***Task*** Use the head() function to quickly have a look at the training data frame.

In [ ]:
#Use the head() function display th first few rows of the training data frame.
train.head()

The testing set is formatted the same as the training set. Notice, that the gene description and accession numbers are given along with the count and outcome (call) for each patient. The patient outcomes are also provided in the file `actual.csv`. I think it will be more convientient to use the outcomes in this file and delete the 'call' columns in both the training and testing sets. ***Question: What are the observational units of interest?*** 

## Formatting
***Task*** Remove the 'call' columns from the training and testing sets.

In [ ]:
#<--remove this for student verssion--> 
cols = [col for col in test.columns if 'call' in col]
test = test.drop(cols, 1)

cols = [col for col in train.columns if 'call' in col]
train = train.drop(cols, 1)

train.head()

Let's consider what the observational unit should be. ***Task*** Format the data to have observations in rows and features in columns.

In [ ]:
# remove this for stutent version 
train = train.T
test = test.T
train.head()

We can also remove the gene bookkeeping data. However, it's probably a good idea to make a copy of the Gene Accession Numbers for later analyses.

In [ ]:
#Save the Gene Accession Numbers
accnum = train[:2]
accnum

In [ ]:
#remove the gene bookkeeping data.
train = train.drop(["Gene Description", "Gene Accession Number"])
test = test.drop(["Gene Description", "Gene Accession Number"])
train.head()

Now let's encode the outcomes for binary classification. We'll use Zeros for the ALL outcomes and Ones for AML. Remember the first 38 patients were partitioned for the training set the remainder are in the testing set.

In [ ]:
labels = labels.replace({'ALL':0, 'AML':1})
labels_train = labels[0:38]
labels_test = labels[38:]

### Treat missing data
Before moving on to a table 1. Let's look for and treat any missing data this. Remember to check for values that don't make sense. I think replacing witht the mean value would be a reasonable imputation strategy.***Task*** check for unreasonable and missing values and impute them with the column mean. 

In [ ]:
#remove inf
train = train.replace( np.inf , np.nan )
test = test.replace( np.inf , np.nan )

#impute with mean
train = train.fillna(value = train.values.mean())
test = test.fillna(value = train.values.mean())

***Question*** How would you go about visualizing the data we just formatted? Why can't I just make a table 1?

### PCA for Dimension Reduction
To motivate the concept, let's project the ~7k dimesions to the first 10 principal components.

In [ ]:
#load preprocessing and PCA from sklearn
from sklearn.decomposition import PCA
from sklearn import preprocessing

#Do a PCA for all data (this should probably be done only with the training data)
nPC=10
pca = PCA(n_components = nPC, random_state = seed)

#define standard scaler
scaler = preprocessing.StandardScaler()

#fit pca on the training data
X_scaled = scaler.fit_transform(train)
X_train_pca = pca.fit_transform(X_scaled)

#transform the testing data using the fit scaler and pca objects
X_test_pca = pca.transform(scaler.transform(test))

print(X_train_pca.shape)

---
**Question:** How many PCs are needed to explain 80% of the variance in the data?

In [ ]:
#load preprocessing and PCA from sklearn
from sklearn.decomposition import PCA
from sklearn import preprocessing

# Full dof PCA for all data
pca = PCA(random_state = seed) #do not define number of PCs

#define standard scaler
scaler = preprocessing.StandardScaler()

#fit pca on the training data
X_scaled = scaler.fit_transform(train)
X_train_pca = pca.fit_transform(X_scaled)

# compute total variation explained
total = sum(pca.explained_variance_)

# init number of PCs to use
nPC = 0

#init variance explained using 0 PCs
cum_variance = 0

#loop while cumulative variance explained is less than 80% of the total
<your code>

#display number of PC chosen
print(nPC, " features explain 80% of the variance. Suggested dimesnion reduction of 7129 features to ", nPC, sep='')

In [ ]:
#Do a PCA with selected number of PCs
pca = PCA(n_components = nPC, random_state = seed)

#fit pca on the training data
X_train_pca = pca.fit_transform(X_scaled)

#transform the testing data using the fit scaler and pca objects
X_test_pca = pca.transform(scaler.transform(test))

print(X_train_pca.shape)

#plot ratio of variance explained by nPCs
var_exp = pca.explained_variance_ratio_.cumsum()
var_exp = var_exp*100
plt.bar(range(1, nPC + 1), var_exp, color="brown")
plt.xlabel("Number of components", fontsize=18)
plt.ylabel("Cumulutive variance explained", fontsize=18)
plt.xlim((0.5, nPC + 1))
plt.show()


### Visualize Engineered Features
Let's plot the feature distributions.

In [ ]:
# rescale the engineered features
X_train_pca = scaler.fit_transform(X_train_pca)
X_test_pca = scaler.transform(X_test_pca)

#plot the principle component distributions
for PC in range(nPC):
    sns.kdeplot(data=X_train_pca[:, PC])
    #X_pca[:,nPC].plot.kde(bw_method='scott') #use bw_method=.02 for a lower bandwidth gaussian representation
    plt.legend(["PC" + str(PC)])
    plt.show()

## LASSO for feature selection

Let's use LASSO to select which of these engineered features to use in our model.

**Question:** Why would LASSO be a good choice for these engineered features? 

In [ ]:
#partitioning disco-train & testing sets
from sklearn.model_selection import train_test_split

# get predictors and labels (ideally use 30% disco set)
X = X_train_pca
y = np.array(labels_train).squeeze()

#split 30% of the the training data off for discovery
X_train, X_disco, y_train, y_disco = train_test_split(X, y, test_size=.3, random_state=seed, shuffle=True, stratify=y)

#get the testing set predictors and labels
X_test = X_test_pca 
y_test = np.array(labels_test).squeeze()


In [ ]:
from sklearn.linear_model import LassoCV

#train lasso model with 5-fold cross validataion using disco set
lasso = <your code>

#plot feature importance based on coeficients
importance = np.abs(lasso.coef_)
feature_names = np.array(range(X_disco.shape[1]))
plt.bar(height=importance, x=feature_names)
plt.xticks(rotation=90)
plt.title("Feature importances via coefficients")
plt.show()

#select PCs with non-zero weight
sel_features = feature_names[importance>0]
print(sel_features)

## Benchmark model Logistic Regression
We'll use logistic regression as our benchmark model.

In [ ]:
#import Logistic Regression model and metric
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix

#define logistic regresssion model and train with training set
benchmark = <your code>

#display performance metrics
display("accuracy = " + str(benchmark.score(X_test, y_test)))
display("f1 score = " + str(f1_score(y_test, benchmark.predict(X_test))))
display(confusion_matrix(y_test, benchmark.predict(X_test)))


## Train Stacked model
Let's train a panel of mesa models and use thier predictions to train a meta model. For this we will use the output of a RF, KNN and support vector machine inputs into a logistic regression meta model.

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

#define mesa models
estimators = [('rf', <your rf classifier>),
              ('knn', <your kn classifier>),
              ('svm', <your svm classifier>)
             ]

#stack mesa models on a logistic regression meta model
stakt = StackingClassifier(estimators=estimators,
                           final_estimator=<your log reg classifier>)

#train stacked model
stakt.fit(X_train, y_train)

#display performance metrics
display("accuracy = " + str(benchmark.score(X_test, y_test)))
display("f1 score = " + str(f1_score(y_test, benchmark.predict(X_test))))
display(confusion_matrix(y_test, benchmark.predict(X_test)))

## Compare ROCs using delongs test.
DeLong is an asymptotically exact method to evaluate the uncertainty of an AUC ([DeLong et al. (1988)](https://www.jstor.org/stable/2531595). It can be used to compare if two AUC's are statistically different. Let's plot the ROCs of the benchmark and stacked models.

In [ ]:
from sklearn.metrics import roc_curve, auc

#get Benchmark class probabilities
preds = benchmark.predict_proba(X_test)[:,1]

#compute ROC curve and AUC
fpr, tpr, _ = <call roc_curve function>
roc_auc = <call auc function>

#plot benchmark ROC curve
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Benchmark receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

#get Model class probabilities
preds = stakt.predict_proba(X_test)[:,1]

#compute ROC curve and AUC
fpr, tpr, _ = roc_curve(y_test, preds)
roc_auc = auc(fpr, tpr)

#plot Model ROC curve
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Stacked Model receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

Fortunately, Yandex Data School has a Fast DeLong implementation on their public repo:
https://github.com/yandexdataschool/roc_comparison

In [ ]:
#copy of deLong's test calculator from Yandex Data School
import pandas as pd
import numpy as np
import scipy.stats

# AUC comparison adapted from
# https://github.com/Netflix/vmaf/
def compute_midrank(x):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5*(i + j - 1)
        i = j
    T2 = np.empty(N, dtype=float)
    # Note(kazeevn) +1 is due to Python using 0-based indexing
    # instead of 1-based in the AUC formula in the paper
    T2[J] = T + 1
    return T2


def fastDeLong(predictions_sorted_transposed, label_1_count):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Operating Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=float)
    ty = np.empty([k, n], dtype=float)
    tz = np.empty([k, m + n], dtype=float)
    for r in range(k):
        tx[r, :] = compute_midrank(positive_examples[r, :])
        ty[r, :] = compute_midrank(negative_examples[r, :])
        tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])
    aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
    v01 = (tz[:, :m] - tx[:, :]) / n
    v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def calc_pvalue(aucs, sigma):
    """Computes log(10) of p-values.
    Args:
       aucs: 1D array of AUCs
       sigma: AUC DeLong covariances
    Returns:
       log10(pvalue)
    """
    l = np.array([[1, -1]])
    z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, sigma), l.T))
    return np.log10(2) + scipy.stats.norm.logsf(z, loc=0, scale=1) / np.log(10)


def compute_ground_truth_statistics(ground_truth):
    assert np.array_equal(np.unique(ground_truth), [0, 1])
    order = (-ground_truth).argsort()
    label_1_count = int(ground_truth.sum())
    return order, label_1_count


def delong_roc_variance(ground_truth, predictions):
    """
    Computes ROC AUC variance for a single set of predictions
    Args:
       ground_truth: np.array of 0 and 1
       predictions: np.array of floats of the probability of being class 1
    """
    order, label_1_count = compute_ground_truth_statistics(ground_truth)
    predictions_sorted_transposed = predictions[np.newaxis, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count)
    assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
    return aucs[0], delongcov


def delong_roc_test(ground_truth, predictions_one, predictions_two):
    """
    Computes log(p-value) for hypothesis that two ROC AUCs are different
    Args:
       ground_truth: np.array of 0 and 1
       predictions_one: predictions of the first model,
          np.array of floats of the probability of being class 1
       predictions_two: predictions of the second model,
          np.array of floats of the probability of being class 1
    """
    order, label_1_count = compute_ground_truth_statistics(ground_truth)
    predictions_sorted_transposed = np.vstack((predictions_one, predictions_two))[:, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count)
    return calc_pvalue(aucs, delongcov)

In [ ]:
#Determine if the benchmark and stacked model AUCs are statistically different

#get ground truth labels
ground_truth = y_test

#get class probabilities
predictions_one = benchmark.predict_proba(X_test)[:,1]
predictions_two = stakt.predict_proba(X_test)[:,1]

#test if two predictions are different
pval = <call delong roc test function>

display("DeLong's AUC test pvalue =" + str(pval))


We can also calculate the confidence intervals on the AUC of the two models to demonstrate if they are different.

In [ ]:
#define significance
alpha = .95

#get bechmark stats
auc, auc_cov = <call delong_roc_variance fucntion>

#compute confidence interval
auc_std = np.sqrt(auc_cov)
lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)

ci = stats.norm.ppf(
    lower_upper_q,
    loc=auc,
    scale=auc_std)

ci[ci > 1] = 1

print('Benchmark AUC:', auc)
print('Benchmark AUC COV:', auc_cov)
print('Benchmark 95% AUC CI:', ci)


In [ ]:
#Repeat CI calculation for stacked model stats  
<your code>

## Interpretations

**Question**: What does it mean if model AUC is the same as that of the benchmark?

**Question**: Does this analysis justify using a more complicated model in a future study that goes beyond logistic regression?

**Question**: What statement about the biology can you make with this analysis?

**Question**: How would you have designed the analysis to improve the AUC?